<a href="https://colab.research.google.com/github/FRemeeus/hashgroup/blob/main/DataAugmentationProject_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and extract dataset

In [ ]:
  # Mounting your google Drive
from google.colab import drive  
drive.mount("/content/drive")

# Copying the dataset
import time
start = time.time()
!cp -n /content/drive/MyDrive/dataset.zip .
end = time.time()
print(f"Time consumed in copying the dataset: {round(end - start, 2)} seconds")

# Unzipping the dataset
start = time.time()
!unzip -nq /content/dataset
end = time.time()
print(f"Time consumed in unzipping: {round(end - start, 2)} seconds")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Time consumed in copying the dataset: 9.38 seconds
Time consumed in unzipping: 5.04 seconds


# Import packages

In [ ]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import WeightedRandomSampler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import seaborn as sns
import matplotlib.pylab as plt
import pickle

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
seed = 42
torch.manual_seed(seed)

PyTorch Version:  1.9.0+cu102
Torchvision Version:  0.10.0+cu102


# Choosing parameters

In [ ]:
# Top level data directory.
data_dir = "/content/dataset/"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "resnet"

# Number of samples per class in training data
num_train = 500

# Number of samples per class in validation data
num_val = 481

# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for 
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model, 
# when True we only update the reshaped layer params
feature_extract = False

# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Code for loading models


In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Training script

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    train_acc_log = []
    val_acc_log = []

    train_loss_log = []
    val_loss_log = []

    confusion_matrix = torch.zeros(num_classes, num_classes)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.

            for i, data in enumerate(dataloaders[phase], 1):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # creating confusion matrix <-- NEW PIECE OF CODE
                    if phase == 'val':
                        for t, p in zip(labels.view(-1), preds.view(-1)):
                            confusion_matrix[t.long(), p.long()] += 1

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                if i % 10 == 1:
                    print(f"Batch {i} / {len(dataloaders[phase])}, Loss: {loss.item()}")

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Storing results of this epoch
            if phase == 'train':
                train_acc_log.append(epoch_acc)
                train_loss_log.append(epoch_loss)
            if phase == 'val':
                val_acc_log.append(epoch_acc)
                val_loss_log.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return train_acc_log, train_loss_log, val_acc_log, val_loss_log, confusion_matrix

# Initialize everything for training

In [ ]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
# finetuning we will be updating all parameters. However, if we are 
# doing feature extract method, we will only update the parameters
# that we have just initialized, i.e. the parameters with requires_grad
# is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

# Loading data and data augmentation


In [ ]:
# Code to add black borders to an image to create a square image
def get_pad(img):
  retlist = np.zeros(2)
  maxie = np.argmin(img.size)
  retlist[maxie] = np.ceil((img.size[maxie-1]-img.size[maxie])/2)
  return tuple(retlist.astype(int))

class ResizePad():

    def __init__(self, size, fill=0, padding_mode='constant'):
      assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']
      self.size = size
      self.fill = fill
      self.padding_mode = padding_mode

    def __call__(self, img):
      pad = transforms.functional.pad(img, get_pad(img), self.fill, self.padding_mode)
      return transforms.CenterCrop(self.size)(transforms.functional.resize(pad, self.size))
    
    def __repr__(self):
      return self.__class__.__name__ + '(fill={0}, padding_mode={1})'.\
            format(self.fill, self.padding_mode)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation

"""
HIER JE AUGMENTATION NEER ZETTEN:
bijvoorbeeld: custom_augmentation = transforms.RandomPerspective()
kiezen uit: None, transforms.RandomPerspective(), transforms.RandomRotation(180), transforms.GaussianBlur(kernel_size=(5,5), sigma=(0.1, 1.0)), 
transforms.RandomVerticalFlip() en transforms.RandomHorizontalFlip().
"""
custom_augmentation = transforms.RandomErasing(p=0.5, scale=(0.02, 0.5), ratio=(0.3, 3.3), value='random')
# custom_augmentation1 = ...
# custom_augmentation2 = ...          <-- uncomment this you if want to stack augmentations


#-------------------------------------------------------------------------------
baseline_transform = transforms.Compose([
  transforms.Resize((input_size, input_size)),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_transforms = {
  'train': transforms.Compose([
      transforms.Resize((input_size, input_size)),
      transforms.ToTensor(),
      custom_augmentation,
      # custom_augmentation1,
      # custom_augmentation2,         <-- uncomment this if you want to stack augmentations
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ]),
  'val': baseline_transform
}

if not custom_augmentation:
  title_label = 0
elif str(custom_augmentation) == "RandomPerspective(p=0.5)":
  title_label = 1
elif str(custom_augmentation) == "RandomRotation(degrees=[-180.0, 180.0], interpolation=nearest, expand=False, fill=0)":
  title_label = 2
elif str(custom_augmentation) == "GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1.0))":
  title_label = 3
elif str(custom_augmentation) == "RandomVerticalFlip(p=0.5)":
  title_label = 4
elif str(custom_augmentation) == "RandomHorizontalFlip(p=0.5)":
  title_label = 5
elif str(custom_augmentation) == "ColorJitter(brightness=(1, 1.5), contrast=None, saturation=None, hue=None)":
  title_label = 6
elif str(custom_augmentation) == "RandomErasing()":
  title_label = 7


print("Initializing Datasets and Dataloaders...")

# Create balanced train and validation sets
def subset(dataset, samples_per_class):
  idxs = []

  last_target = -1
  for i, target in enumerate(dataset.targets):
    if target != last_target:
      for j in range(samples_per_class):
        idxs.append(i + j)
    last_target = target

  return torch.utils.data.Subset(dataset, idxs)

val_data = subset(datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val']), num_val)
baseline = datasets.ImageFolder(os.path.join(data_dir, 'train'), baseline_transform)

double = False
if custom_augmentation:
  if double:
    add_on = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
    train_data = torch.utils.data.ConcatDataset([subset(baseline, num_train), subset(add_on, num_train)])
  else:
    train_data = subset(datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']), num_train)
else:   
  train_data = subset(baseline, num_train)    

dataloaders_dict = {}
dataloaders_dict['val'] = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=2, shuffle=True)
dataloaders_dict['train'] = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=2, shuffle=True)

RandomErasing(p=0.5, scale=(0.02, 0.5), ratio=(0.3, 3.3), value=random, inplace=False)
Initializing Datasets and Dataloaders...


# Train and evaluate the model

In [ ]:
train_acc_log, train_loss_log, val_acc_log, val_loss_log, confusion_matrix = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)


NameError: ignored

In [ ]:
# code to save the runs into a folder
def list_to_cpu(tensor_list):
  lst = []
  for element in tensor_list:
    lst.append(element.to(torch.device('cpu')))
  return lst

pickle.dump([list_to_cpu(train_acc_log), train_loss_log, list_to_cpu(val_acc_log), val_loss_log, confusion_matrix], 
            open("drive/MyDrive/roaddefects/log_param/" + str(model_name) + "§" + str(custom_augmentation) + "§" + str(seed) + ".txt", "wb" ))